In [ ]:
import numpy as np
import copy
from tqdm import trange

In [ ]:
def gravity(m1, m2):
    shift = [0,0,0]
    for a in range(0,3):
        p1 = m1["pos"][a]
        p2 = m2["pos"][a]
        
        if p1<p2:
            shift[a] = 1
        elif p2<p1:
            shift[a] = -1

    m1["velocity"] = np.add(m1["velocity"], shift)
    m2["velocity"] = np.add(m2["velocity"], np.multiply(shift, -1))
            
def velocity(moon):
    moon["pos"] = np.add(moon["pos"], moon["velocity"])
    
def energy(moon):
    p = np.sum(np.abs(moon["pos"]))
    k = np.sum(np.abs(moon["velocity"]))
    moon["energy"] = k*p

def print_debug(moons, i):
    print("Iteration", i)
    e = 0
    for m in moons:
        print("  p={:3},{:3},{:3} v={:3},{:3},{:3} E={}".format(*m["pos"],*m["velocity"], m.get("energy",0)) )
        e += m.get("energy",0)
    print("  E: {}".format(e))
    
    
def model(moons, iterations, debug=None):
    moons = copy.deepcopy(moons)
    for i in range(0, iterations):
        for a in range(0, len(moons)):
            for b in range(a+1, len(moons)):
                gravity(moons[a], moons[b])

        for a in range(0, len(moons)):
            velocity(moons[a])
            energy(moons[a])
            
        
        if debug is not None and (i+1)%debug==0:
            print_debug(moons, i+1)

    print_debug(moons, "END")

        
def create_moon(x,y,z):
    return {
        "pos": [x,y,z],
        "velocity": [0,0,0]
    }


In [ ]:
import re
def parse_moon(value):
    m = re.search("x=(.*?), y=(.*?), z=(.*?)>",value)
    (x,y,z) = m.groups()
    return {
        "pos": [int(x),int(y),int(z)],
        "velocity": [0,0,0]
    }
    

In [ ]:
sample1 = """
<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
""".splitlines()
sample1 = [parse_moon(x) for x in sample1 if len(x)>0]
model(sample1, 10, debug=1)

In [ ]:
sample2 = """
<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
""".splitlines()
sample2 = [parse_moon(x) for x in sample2 if len(x)>0]
model(sample2, 100, debug=10)

In [ ]:
with open("12-input.txt", "rt") as FILE:
    data = FILE.read().splitlines()
data = [parse_moon(x) for x in data if len(x)>0]
model(data, 1000)

# Part 2

In [ ]:
def model_repeat(moons, iterations, debug=None):
    initial = moons
    moons = copy.deepcopy(moons)

    for i in trange(0, iterations):
        for a in range(0, len(moons)):
            for b in range(a+1, len(moons)):
                gravity(moons[a], moons[b])

        for a in range(0, len(moons)):
            velocity(moons[a])
            energy(moons[a])
        
        if debug is not None and (i+1)%debug==0:
            print_debug(moons, i+1)
            
        e = np.sum([np.abs(m["velocity"]) for m in moons])
        if (e == 0):
            if list(moons[0]["pos"]) == list(initial[0]["pos"]):
                print(i+1, moons)
                return

        



In [ ]:
model_repeat(sample1, 10000)

# Part 2 - Take 2

I had to look this one up... :-(

Basically, each dimension only depends on itself - we can therefore consider each dimension separately and find when each dimension loops. We then simply need to find the LCM of all the dimensions' periods.

In [ ]:
def check_match(initial, moons, d):
    for m in range(0, len(moons)):
        if moons[m]["pos"][d] != initial[m]["pos"][d]:
            return False
        if moons[m]["velocity"][d] != initial[m]["velocity"][d]:
            return False
    return True
    
def find_cycles(moons, iterations, debug=None):
    initial = moons
    moons = copy.deepcopy(moons)

    cycles = [None, None, None]

    for i in range(0,iterations):
        for a in range(0, len(moons)):
            for b in range(a+1, len(moons)):
                gravity(moons[a], moons[b])

        for a in range(0, len(moons)):
            velocity(moons[a])            
            

        for d in range(0,3):
            if cycles[d]:
                continue
            if check_match(initial, moons, d):
                cycles[d] = i+1
                print("Found cycle for d={}: {}".format(d, i))
                count = sum(1 for e in cycles if e)
                if count >= 3:
                    return cycles 

                          
        if debug is not None and (i+1)%debug==0:
            print_debug(moons, i+1)
    
    


In [ ]:
cycles = find_cycles(sample1, 100000000)
np.lcm.reduce(cycles)

In [ ]:
cycles = find_cycles(sample2, 100000000)
np.lcm.reduce(cycles)

In [ ]:
cycles = find_cycles(data, 100000000)
np.lcm.reduce(cycles)